# Machine Learning model - Regression 
For year 2014-2015 World GDP Prediction


In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

text = sc.textFile('filled.csv')
text = text.map(lambda text:text.split(";"))


countries = ['Canada','India','United States','China','France']
header = text.first()
features = text.filter(lambda line:line!=header)

for country in countries:
    data = features.filter(lambda text:text[-1]==country)
    data = data.filter(lambda text:text[-3]>1984)
    test = data.filter(lambda text:text[-3] in ['2014','2015'])
    train = data.filter(lambda text:text[-3] not in ['2014','2015'])
    train = train.map(lambda text:(float(text[-2]),Vectors.dense(text[0:3]))).toDF(['label','features'])
    test = test.map(lambda text:(float(text[-2]),Vectors.dense(text[0:3]))).toDF(['label','features'])
    lr = LinearRegression(maxIter=3)
    pipeline = Pipeline(stages=[lr])
    paramGrid = (ParamGridBuilder().addGrid(lr.regParam, [1, 0.75 ,0.5, 0.1, 0.01])
                                   .addGrid(lr.elasticNetParam, [0.0,0.5, 1.0])
                                   .build())
    crossval = CrossValidator(estimator=pipeline,
                              estimatorParamMaps=paramGrid,
                              evaluator=RegressionEvaluator(),
                              numFolds=3)
    cvModel = crossval.fit(train)
    prediction = cvModel.transform(test)
    prediction.show()

+-----------+--------------------+------------------+
|      label|            features|        prediction|
+-----------+--------------------+------------------+
|2.472892638|[68.2722,1.237576...|1.7333235964125357|
|1.078268751|[67.8922,1.253367...|1.7216043513520276|
+-----------+--------------------+------------------+

+-----------+--------------------+-----------------+
|      label|            features|       prediction|
+-----------+--------------------+-----------------+
|7.243471746|[38285.056,141.87...| 8.08099797760533|
|7.570130368|[37762.815,141.87...|8.106384150562933|
+-----------+--------------------+-----------------+

+-----------+--------------------+------------------+
|      label|            features|        prediction|
+-----------+--------------------+------------------+
|2.427795636|[5.0,44.221248625...|1.9496426632383344|
|2.425970526|[5.0,44.618038175...|0.9735467793007544|
+-----------+--------------------+------------------+

+-----------+------------------